In [1]:
%load_ext autoreload
%autoreload 2

from DQN import DQN
import tensorflow as tf
import numpy as np
import keras


In [ ]:
myModel = DQN(8, 4, 64)

inputData = tf.ones((1,8))
print(inputData)
y = myModel(inputData)

# myModel.compile(
#      loss = 'mse',
#      metrics = ['mse'],
#      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
myModel.summary()
# print(y)

tf.Tensor([[1. 1. 1. 1. 1. 1. 1. 1.]], shape=(1, 8), dtype=float32)


Model: "dqn_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (1, 64)                │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (1, 64)                │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (1, 4)                 │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,996 (19.52 KB)

 Trainable params: 4,996 (19.52 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
from lunar import LunarLanderEnv

lunar = LunarLanderEnv(render_mode=None)
print(lunar.env.observation_space)
print(lunar.env.action_space.sample())
print(lunar.state)

Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
3
[-0.00701618  1.4053609  -0.71068066 -0.24709083  0.00813681  0.1609798
  0.          0.        ]


c:\Users\miniluz\AppData\Local\pypoetry\Cache\virtualenvs\aprendizaje-modulo-lunar-5laruUkD-py3.10\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [4]:
lunarState = lunar.state
lunarTensor = tf.convert_to_tensor([lunarState], dtype=np.float32)
result = myModel(lunarTensor)
print(np.argmax(result[0]))
print(result[0][1])

3
tf.Tensor(-0.02254521, shape=(), dtype=float32)


In [5]:
lunarState = lunar.state
lunarTensor = tf.convert_to_tensor([lunarState] * 100, dtype=np.float32)
actions = tf.convert_to_tensor([0, 1, 0, 3] * 25, dtype=np.int32)
# print(tf.one_hot(actions, 4))
print(myModel(lunarTensor) * tf.one_hot(actions, 4))
result = tf.reduce_sum(myModel(lunarTensor) * tf.one_hot(actions, 4), 1)
print(result)

tf.Tensor(
[[-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.   

In [6]:
lunarState = lunar.state
states = tf.convert_to_tensor([lunarState] * 100, dtype=np.float32)
next_states =  tf.convert_to_tensor([lunarState] * 100, dtype=np.float32)
rewards =  tf.convert_to_tensor([1] * 100, dtype=np.float32)
dones =  tf.convert_to_tensor([0] * 100, dtype=np.float32)
optimizer = keras.optimizers.Adam(learning_rate=0.01)

with tf.GradientTape() as tape:
    # Get Q-values for the actions that were actually taken
    q_values = tf.reduce_sum(myModel(states) * tf.one_hot(actions, 4), axis=1)
    
    # Get maximum Q-value for the next states from target network
    next_q_values = tf.reduce_max(myModel(next_states), axis=1)
    
    # Compute the expected Q-values
    expected_q_values = rewards + 0.9 * next_q_values * (1 - dones)
    
    # Compute the loss between the current and expected Q values
    loss = tf.keras.losses.MeanSquaredError()(expected_q_values, q_values)

gradients = tape.gradient(loss, myModel.trainable_weights)
for gradient in gradients:
    print(gradient.shape)

optimizer.apply_gradients(zip(gradients, myModel.trainable_weights))

(8, 64)
(64,)
(64, 64)
(64,)
(64, 4)
(4,)


<Variable path=adam/iteration, shape=(), dtype=int64, value=1>